In [ ]:
%matplotlib inline

## YOLO

In [ ]:
from src.hpe.common.cleanup import remove_temp_factory
from src.hpe.yolo.mutators import adapt_to_bgr
from src.hpe.yolo.performance import estimate_distances

In [ ]:
# default estimation
distances = estimate_distances()

In [ ]:
# estimation on bgr images
distances_bgr = estimate_distances(
    dataset_name="distances_bgr",
    image_path_mutators=[adapt_to_bgr],
    clean_up=[remove_temp_factory()]
)

In [ ]:
from src.hpe.yolo.performance import read_distances, estimate_performance
from src.hpe.yolo.plot import plot_yolo_average_distances

In [ ]:
estimate_performance(name="Yolov11m-pose")

In [ ]:
distances = read_distances()
plot_yolo_average_distances(distances=distances)

In [ ]:
# calculate and log performance on bgr images
estimate_performance(
    name="Yolov11m-pose on bgr",
    image_path_mutators=[adapt_to_bgr],
    clean_up=[remove_temp_factory()]
)

In [ ]:
distances_bgr = read_distances(dataset_name="distances_bgr")
plot_yolo_average_distances(distances=distances_bgr)

In [ ]:
from src.hpe.yolo.play import play_with_hpe as play_with_yolo_hpe
from glob import glob
from random import choice

video_paths = glob("data/samples/**/*.*")
video_path = choice(video_paths)

play_with_yolo_hpe(video_path)

## MediaPipe

In [1]:
from src.hpe.mp.performance import estimate_distances
from src.hpe.mp.mutators import convert_to_bgr

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-09-30 11:37:25.398221: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


loaded labels


In [2]:
distances = estimate_distances()

libEGL warning: DRI3: Screen seems not DRI3 capable
libEGL warning: DRI3: Screen seems not DRI3 capable
MESA: error: ZINK: failed to choose pdev
libEGL warning: egl: failed to create dri2 screen
I0000 00:00:1759225100.993922  113516 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1759225101.047583  113948 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: D3D12 (NVIDIA TITAN RTX)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1759225101.641749  113874 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759225104.768594  113869 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759225104.792333  113932 inference_feedback_manager.cc:114] Feedback manager requires a model with a single s

Distances saved to 'data/hpe/mediapipe/distances.pkl'


In [3]:
# estimation on bgr images
distances_bgr = estimate_distances(
    dataset_name="distances_bgr",
    image_mutators=[convert_to_bgr]
)

I0000 00:00:1759225655.557350  113516 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1759225655.583413  115669 gl_context.cc:357] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.2.8-1ubuntu1~24.04.1), renderer: D3D12 (NVIDIA TITAN RTX)
W0000 00:00:1759225655.868959  115606 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759225656.931287  115604 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759225656.943155  115615 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1759225656.945277  115614 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:

Distances saved to 'data/hpe/mediapipe/distances_bgr.pkl'


In [ ]:
from src.hpe.mp.performance import estimate_performance

In [ ]:
estimate_performance(dataset_name="MediaPipe")

In [ ]:
estimate_performance(dataset_name="MediaPipe on bgr",
    image_mutators=[convert_to_bgr])

In [ ]:
from src.hpe.mp.performance import read_distances
from src.hpe.mp.plot import plot_mediapipe_average_distances

In [ ]:
distances = read_distances()
plot_mediapipe_average_distances(distances=distances)

In [ ]:
distances_bgr = read_distances(dataset_name="distances_bgr")
plot_mediapipe_average_distances(distances=distances_bgr)

In [ ]:
from src.hpe.mp.play import play_with_hpe
from glob import glob
from random import choice

video_paths = glob("data/samples/**/*.*")
video_path = choice(video_paths)

play_with_hpe(video_path)

## Comparison

In [ ]:
from src.hpe.yolo.performance import read_distances as read_yolo_distances
from src.hpe.mp.performance import read_distances as read_mp_distances
from src.hpe.common.plot import plot_distances_boxplot

In [ ]:
mp_dist = read_mp_distances(dataset_name="distances")
mp_bgr_dist = read_mp_distances(dataset_name="distances_bgr")

yolo_dist = read_yolo_distances(dataset_name="distances")
yolo_bgr_dist = read_yolo_distances(dataset_name="distances_bgr")


In [ ]:
plot_distances_boxplot(
    None,
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))

In [ ]:
plot_distances_boxplot(
    (0, 10),
    ("MediaPipe", mp_dist),
    ("MediaPipe on bgr", mp_bgr_dist),
    ("Yolov11", yolo_dist),
    ("Yolov11 on bgr", yolo_bgr_dist))